In [93]:
from sklearn.decomposition import TruncatedSVD, randomized_svd
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [104]:
#user = pd.read_csv('./whyout_data/user.csv') # shape(31178,13), user idx에서 2354개가 비어있음
#place = pd.read_csv('./whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
product = pd.read_csv('./whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
#video = pd.read_csv('./whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
#user_place = pd.read_csv('./whyout_data/user_place.csv', index_col=0).astype(int) # shape (31176,4697) 유저 x 장소 아이템 # 8756명이 장소를 평가하지 않음
user_product = pd.read_csv('./whyout_data/user_product.csv', index_col=0).astype(int) # shape(31176,5834) 유저 x 상품 아이템
#user_video = pd.read_csv('./whyout_data/user_video.csv', index_col=0).astype(int) # shape (31177, 3250) 유저 x 영상 아이템

In [96]:
# 데이터프레임을 numpy matrix로 만듦
user_product_matrix = user_product.values

# 사용자의 평균 평점
user_ratings_mean = np.mean(user_product_matrix, axis=1)

# 사용자-상품에 대해 사용자 평균 평점을 뺀 것
matrix_user_mean = user_product_matrix - user_ratings_mean.reshape(-1,1)

In [97]:
pd.DataFrame(matrix_user_mean, columns = user_product.columns).head()

,1,2,3,4,5,6,7,8,9,10,...,5824,5825,5826,5827,5828,5829,5830,5831,5832,5833
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,...,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514


In [98]:
# U 행렬, sigma 행렬, V 전치 행렬을 반환.
# U, sigma, Vt = svds(matrix, k = 20)
U, sigma, Vt = randomized_svd(user_product_matrix, n_components=10)

# 특이값 벡터를 대각 행렬 형태로 변환해줌
sigma = np.diag(sigma)

In [99]:
# U, Sigma, Vt의 행렬곱을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.matmul(np.matmul(U,sigma), Vt) + user_ratings_mean.reshape(-1,1)

In [100]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = user_product.columns)

In [101]:
product

,idx,카테고리1,카테고리2,색상,기능성(합집합),휴대성,친환경,디자인,클릭 수,좋아요 수,북마크 수
0,2,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.000000,0.017857
1,3,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.015873,0.000000
2,4,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.015873,0.017857
3,5,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.000000,0.000000
4,6,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]]","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.015873,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5829,6369,"[0,0,1,0,0,0,0,0,0,0,0]","[0,0,1]","[[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0],[0,1,0,0,0,0]]","[[0,1,0,0]]","[[1,0,0],[0,1,0]]",NaN,NaN,0.000000,0.000000
5830,6370,"[0,0,1,0,0,0,0,0,0,0,0]","[0,0,1]","[[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[[1,0,0,0,0,0]]","[[1,0,0,0]]","[[0,1,0]]",NaN,NaN,0.000000,0.000000
5831,6371,"[0,1,0,0,0,0,0,0,0,0,0]","[0,1,0]",NaN,"[[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0]]",NaN,"[[1,0,0]]",NaN,NaN,0.000000,0.000000
5832,6372,"[0,1,0,0,0,0,0,0,0,0,0]","[0,1,0]",NaN,"[[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0]]",NaN,"[[1,0,0]]",NaN,NaN,0.000000,0.000000


In [144]:
user_product

,1,2,3,4,5,6,7,8,9,10,...,5824,5825,5826,5827,5828,5829,5830,5831,5832,5833
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31172,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31174,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
df_svd_preds

,1,2,3,4,5,6,7,8,9,10,...,5824,5825,5826,5827,5828,5829,5830,5831,5832,5833
0,-1.913428e-16,-1.520287e-18,-3.393115e-16,-9.284866e-18,-3.413429e-17,-1.148341e-16,3.101555e-30,-8.065266e-30,-1.317334e-30,-4.391390e-32,...,9.386808e-20,0.000000,2.402329e-21,0.000000,5.339462e-35,5.339462e-35,0.000000,8.919417e-21,3.485036e-24,1.925429e-17
1,3.084364e-16,1.342336e-18,-4.383020e-17,1.554990e-17,1.483590e-17,5.123044e-18,2.097948e-30,4.855160e-31,-5.046216e-30,-1.656927e-30,...,4.427380e-20,0.000000,-1.868659e-21,0.000000,6.705335e-34,6.705335e-34,0.000000,-9.477202e-21,-3.082966e-24,8.420928e-17
2,2.382431e-16,2.576698e-18,8.052937e-16,3.850525e-17,7.080360e-17,2.603627e-16,1.578260e-30,-5.637035e-30,-6.301060e-30,-3.083671e-30,...,3.231041e-18,0.000000,-3.572933e-21,0.000000,1.178582e-33,1.178582e-33,0.000000,5.169098e-20,2.022294e-23,1.436888e-16
3,4.435412e-17,2.369941e-18,1.689444e-16,-1.121757e-17,1.149799e-17,5.428251e-17,-1.481163e-30,1.818564e-30,-1.370813e-30,-1.204873e-30,...,1.544615e-19,0.000000,3.579774e-21,0.000000,4.112359e-34,4.112359e-34,0.000000,7.026406e-21,2.664025e-24,2.655328e-17
4,5.143429e-04,5.143162e-04,5.143320e-04,5.143154e-04,5.143169e-04,5.143210e-04,5.143151e-04,5.143151e-04,5.143151e-04,5.143151e-04,...,5.143154e-04,0.000514,5.143151e-04,0.000514,5.143151e-04,5.143151e-04,0.000514,5.143151e-04,5.143151e-04,5.143222e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31171,5.149236e-03,5.143203e-03,5.143758e-03,5.143038e-03,5.143518e-03,5.143654e-03,5.143151e-03,5.143151e-03,5.143151e-03,5.143151e-03,...,5.143161e-03,0.005143,5.143151e-03,0.005143,5.143151e-03,5.143151e-03,0.005143,5.143151e-03,5.143151e-03,5.143078e-03
31172,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
31173,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
31174,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00


In [151]:
df_svd_preds.head(7)

,1,2,3,4,5,6,7,8,9,10,...,5824,5825,5826,5827,5828,5829,5830,5831,5832,5833
0,-1.913428e-16,-1.520287e-18,-3.393115e-16,-9.284866e-18,-3.413429e-17,-1.148341e-16,3.101555e-30,-8.065266e-30,-1.317334e-30,-4.391390e-32,...,9.386808e-20,0.000000,2.402329e-21,0.000000,5.339462e-35,5.339462e-35,0.000000,8.919417e-21,3.485036e-24,1.925429e-17
1,3.084364e-16,1.342336e-18,-4.383020e-17,1.554990e-17,1.483590e-17,5.123044e-18,2.097948e-30,4.855160e-31,-5.046216e-30,-1.656927e-30,...,4.427380e-20,0.000000,-1.868659e-21,0.000000,6.705335e-34,6.705335e-34,0.000000,-9.477202e-21,-3.082966e-24,8.420928e-17
2,2.382431e-16,2.576698e-18,8.052937e-16,3.850525e-17,7.080360e-17,2.603627e-16,1.578260e-30,-5.637035e-30,-6.301060e-30,-3.083671e-30,...,3.231041e-18,0.000000,-3.572933e-21,0.000000,1.178582e-33,1.178582e-33,0.000000,5.169098e-20,2.022294e-23,1.436888e-16
3,4.435412e-17,2.369941e-18,1.689444e-16,-1.121757e-17,1.149799e-17,5.428251e-17,-1.481163e-30,1.818564e-30,-1.370813e-30,-1.204873e-30,...,1.544615e-19,0.000000,3.579774e-21,0.000000,4.112359e-34,4.112359e-34,0.000000,7.026406e-21,2.664025e-24,2.655328e-17
4,5.143429e-04,5.143162e-04,5.143320e-04,5.143154e-04,5.143169e-04,5.143210e-04,5.143151e-04,5.143151e-04,5.143151e-04,5.143151e-04,...,5.143154e-04,0.000514,5.143151e-04,0.000514,5.143151e-04,5.143151e-04,0.000514,5.143151e-04,5.143151e-04,5.143222e-04
5,-1.163924e-18,1.131501e-18,5.809315e-17,-9.105185e-18,2.270275e-18,1.684524e-17,3.914065e-31,-3.857547e-30,6.161402e-31,3.265492e-31,...,2.442423e-19,0.000000,1.444296e-21,0.000000,-1.616020e-34,-1.616020e-34,0.000000,8.543453e-21,3.141593e-24,-2.650108e-18
6,5.355633e-17,2.266718e-18,-1.822839e-16,-2.167407e-18,-1.148623e-17,-5.568845e-17,-1.295555e-30,-3.017320e-30,-5.576108e-31,-1.386720e-30,...,4.491871e-19,0.000000,2.829361e-21,0.000000,4.579595e-34,4.579595e-34,0.000000,1.625316e-20,7.089215e-24,1.787675e-17


In [175]:
def recommend_movies(df_svd_preds, user_id, df, ratings_df, num_recommendations):
    
    # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
    user_data = ratings_df.loc[user_id]  # Series 형태로 가져오기
    #print(user_data)

    # 사용자가 이미 평가한 상품의 인덱스를 추출
    user_history_indices = user_data[user_data > 0].index.tolist()
    print(user_history_indices)

    # SVD를 통해 예측된 사용자의 상품 평점을 기반으로 상품 데이터를 정렬
    # 현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id - 1
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)

    # 사용자가 아직 평가하지 않은 상품 목록을 추출
    recommendations = df.loc[~df.index.isin(user_data[user_history_indices])]
    print(len(recommendations))
    print(len(df))

    # 예측된 상품 평점 데이터와 원본 상품 데이터를 합침
    predictions_df = pd.DataFrame(sorted_user_predictions).reset_index()
    predictions_df['index'] = predictions_df['index'].astype(int)
    recommendations = recommendations.merge(predictions_df, left_index=True, right_on='index')

    # 컬럼 이름을 바꾸고 정렬해서 반환
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations]

    return user_history_indices, recommendations

# 함수 사용 예시
already_rated, predictions = recommend_movies(df_svd_preds, 5, product, user_product, 5)

['518']
5833
5834


In [174]:
predictions

,idx,카테고리1,카테고리2,색상,기능성(합집합),휴대성,친환경,디자인,클릭 수,좋아요 수,북마크 수,index,Predictions
0,3880,"[0,0,0,0,0,0,0,0,1,0,0]","[0,1,0]","[[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]]","[[0,0,1,0,0,0]]","[[0,0,1,0]]","[[0,1,0]]","[[0,1,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0],[...",NaN,0.142857,0.125000,3579,0.000525
1,1375,"[0,0,0,0,0,0,0,0,0,0,1]","[0,0,1]","[[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0],[0,0,1,0,0,0]]","[[0,0,1,0]]","[[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.063492,0.071429,1201,0.000520
2,3520,"[0,0,1,0,0,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]]",NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,3296,0.000518
3,3831,"[0,1,0,0,0,0,0,0,0,0,0]","[1,0,0]","[[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,NaN,NaN,NaN,0.079365,0.071429,3530,0.000517
4,1373,"[0,0,0,0,0,0,0,0,0,0,1]","[0,0,1]","[[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[[0,0,1,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0],[...",NaN,0.063492,0.250000,1199,0.000517
